In [2]:
import numpy as np

import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import pickle
import os
from datetime import datetime

In [3]:
import tensorflow as tf

# Reading the photos

In [4]:
label_dic = {
    'cow' : 0,
    'horse' : 1,
    'pig' : 2,
    'chicken' : 3
}

In [10]:
image_set = {}
resizing = (32, 32)

for label in label_dic:
    image_dir = os.getcwd() + '/' + label
    labeled_image = []

    os.walk(image_dir)

In [5]:
image_set = {}
resizing = (32, 32)

for label in label_dic:
    image_dir = os.getcwd() + '/' + label
    labeled_image = []

    for path, dir, files in os.walk(image_dir):
        for file in files:
            image_dir = path + '/' + file
            img = Image.open(image_dir)
            img = img.resize(resizing)
            if not img.format == "RGB": # 이미지의 포맷이 RGB가 아닐 경우, RGB로 convert 시킴
                img = img.convert("RGB")
            labeled_image.append(np.array(img))
    
    image_set[label] = np.array(labeled_image)

In [5]:
train_class_size = 300
test_class_size = 50


data_set = {
    'train_image': np.empty(1),
    'train_label': np.empty(1),
    'test_image': np.empty(1),
    'test_label': np.empty(1),
}

In [6]:
train_class_size = 300
test_class_size = 50

data_set = {
    'train_image' : np.empty((1,resizing[0],resizing[1],3)),
    'train_label' : np.empty(1),
    'test_image' : np.empty((1,resizing[0],resizing[1],3)),
    'test_label' : np.empty(1)
}

for label in image_set:
    label_index = np.random.randint(len(image_set[label]), size=train_class_size + test_class_size)
    
    train_image = image_set[label][label_index[:train_class_size]]
    test_image = image_set[label][label_index[train_class_size:train_class_size+test_class_size]]
    train_label = np.repeat(label_dic[label], train_class_size)
    test_label = np.repeat(label_dic[label], test_class_size)
    
    data_set['train_image'] = np.concatenate((data_set['train_image'], train_image), axis = 0)
    data_set['test_image'] = np.concatenate((data_set['test_image'], test_image), axis = 0)
    data_set['train_label'] = np.concatenate((data_set['train_label'], train_label), axis = 0)
    data_set['test_label'] = np.concatenate((data_set['test_label'], test_label), axis = 0) # np.concatenate는 cbind, rbind 같은 기능

data_set['train_image'] = data_set['train_image'][1:]
data_set['train_label'] = data_set['train_label'][1:]
data_set['test_image'] = data_set['test_image'][1:]
data_set['test_label'] = data_set['test_label'][1:] # 처음에 넣어줬던 무의미한 값을 빼주는 단계 

In [7]:
data_set['train_image'].shape, data_set['test_image'].shape

((1200, 32, 32, 3), (200, 32, 32, 3))

In [8]:
data_set['train_label'].shape, data_set['test_label'].shape

((1200,), (200,))

# Tensorflow get started !!!!

#### Input - [Conv-ReLU-Pool] - FC -Output

In [19]:
tf.reset_default_graph()

In [20]:
x = tf.placeholder(dtype=tf.float32, shape=[None,32,32,3]) # 이미지 데이터의 경우 shape은 [number, height, width, channels] 순이다
y = tf.placeholder(dtype=tf.int32, shape=[None,1])
y_one_hot = tf.one_hot(y, 4)
y_one_hot = tf.reshape(y_one_hot, [-1, 4])

In [21]:
with tf.name_scope("First_Layer") as scope: 
    w1 = tf.Variable(tf.random_normal(shape=[3,3,3,32], stddev=0.01), name = "W1") # shape !!!!!!!!!!
    L1 = tf.nn.conv2d(input= x, filter= w1, strides=[1,1,1,1], padding='SAME', name="L1")
    
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1],  strides= [1,2,2,1], padding='SAME')

In [22]:
with tf.name_scope("Logits") as scope:
    L1_flat = tf.reshape(L1, shape=[-1,16*16*32]) 
    w2 = tf.get_variable(shape=[16*16*32,4], initializer= tf.contrib.layers.xavier_initializer(), name = "w2") 
    b = tf.Variable(tf.random_normal(shape = [4]))
    logits = tf.matmul(L1_flat , w2) + b

AttributeError: module 'pandas' has no attribute 'computation'

In [ ]:
with tf.name_scope("Training") as scope:
    xentrophy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_one_hot)
    loss = tf.reduce_mean(input_tensor=xentrophy, name= "loss")
    optimizer = tf.train.AdamOptimizer(learning_rate= 0.001)
    training_op = optimizer.minimize(loss)
    
init = tf.global_variables_initializer()
correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
train_y = np.reshape(data_set['train_label'], newshape=[-1,1])
test_y = np.reshape(data_set['test_label'], newshape=[-1,1])

In [ ]:
n_epochs = 15
batch_size = 20

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(data_set['train_image'].shape[0]//batch_size):
            index = np.random.randint(train_class_size*4 ,size = batch_size)
            sess.run(training_op, feed_dict={x : data_set['train_image'][index], y : train_y[index]})
            
        acc_train = accuracy.eval(feed_dict={x : data_set['train_image'][index], y : train_y[index]})
        acc_test = accuracy.eval(feed_dict={x : data_set['test_image'], y : test_y})
        print("Epoch:", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

###   Doing [Conv-ReLU-Pool] one more time again!

In [113]:
with tf.name_scope("Second_Layer") as scope:  
    w2 = tf.Variable(tf.random_normal(shape=[3,3,32,64], stddev=0.01))

    L2 = tf.nn.conv2d(input= L1, filter= w2, strides=[1,1,1,1], padding='SAME') 
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1],  strides= [1,2,2,1], padding='SAME')

In [114]:
with tf.name_scope("Logits") as scope:
    L2_flat = tf.reshape(L2, shape=[-1,8*8*64]) 
    w3 = tf.get_variable(shape=[8*8*64,4], initializer= tf.contrib.layers.xavier_initializer(), name = "w3")
    b = tf.Variable(tf.random_normal(shape = [4]))
    logits = tf.matmul(L2_flat , w3) + b

In [115]:
with tf.name_scope("Training") as scope:
    xentrophy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_one_hot)
    loss = tf.reduce_mean(input_tensor=xentrophy, name= "loss")
    optimizer = tf.train.AdamOptimizer(learning_rate= 0.001)
    training_op = optimizer.minimize(loss)
    
init = tf.global_variables_initializer()
correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y_one_hot, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [116]:
n_epochs = 15
batch_size = 20

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(data_set['train_image'].shape[0]//batch_size):
            index = np.random.randint(train_class_size*4 ,size = batch_size)
            sess.run(training_op, feed_dict={x : data_set['train_image'][index], y : train_y[index]})
            
        acc_train = accuracy.eval(feed_dict={x : data_set['train_image'][index], y : train_y[index]})
        acc_test = accuracy.eval(feed_dict={x : data_set['test_image'], y : test_y})
        print("Epoch:", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

Epoch: 0 Train accuracy: 0.85 Test accuracy: 0.75
Epoch: 1 Train accuracy: 0.95 Test accuracy: 0.785
Epoch: 2 Train accuracy: 0.85 Test accuracy: 0.8
Epoch: 3 Train accuracy: 0.95 Test accuracy: 0.825
Epoch: 4 Train accuracy: 1.0 Test accuracy: 0.84
Epoch: 5 Train accuracy: 1.0 Test accuracy: 0.845
Epoch: 6 Train accuracy: 1.0 Test accuracy: 0.845
Epoch: 7 Train accuracy: 1.0 Test accuracy: 0.88
Epoch: 8 Train accuracy: 1.0 Test accuracy: 0.88
Epoch: 9 Train accuracy: 1.0 Test accuracy: 0.89
Epoch: 10 Train accuracy: 1.0 Test accuracy: 0.89
Epoch: 11 Train accuracy: 1.0 Test accuracy: 0.89
Epoch: 12 Train accuracy: 1.0 Test accuracy: 0.885
Epoch: 13 Train accuracy: 1.0 Test accuracy: 0.89
Epoch: 14 Train accuracy: 1.0 Test accuracy: 0.89


###  DROP OUT !

In [ ]:
L1 = tf.nn.dropout(L1, keep_prob=0.7)
L2 = tf.nn.dropout(L2, keep_prob=0.7)

In [93]:
n_epochs = 15
batch_size = 20

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(data_set['train_image'].shape[0]//batch_size):
            index = np.random.randint(train_class_size*4 ,size = batch_size)
            sess.run(training_op, feed_dict={x : data_set['train_image'][index], y : train_y[index]})
            
        acc_train = accuracy.eval(feed_dict={x : data_set['train_image'][index], y : train_y[index]})
        acc_test = accuracy.eval(feed_dict={x : data_set['test_image'], y : test_y})
        print("Epoch:", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

Epoch: 0 Train accuracy: 0.65 Test accuracy: 0.56
Epoch: 1 Train accuracy: 0.75 Test accuracy: 0.755
Epoch: 2 Train accuracy: 0.95 Test accuracy: 0.8
Epoch: 3 Train accuracy: 1.0 Test accuracy: 0.835
Epoch: 4 Train accuracy: 1.0 Test accuracy: 0.795
Epoch: 5 Train accuracy: 1.0 Test accuracy: 0.825
Epoch: 6 Train accuracy: 1.0 Test accuracy: 0.825
Epoch: 7 Train accuracy: 0.95 Test accuracy: 0.83
Epoch: 8 Train accuracy: 1.0 Test accuracy: 0.84
Epoch: 9 Train accuracy: 1.0 Test accuracy: 0.885
Epoch: 10 Train accuracy: 1.0 Test accuracy: 0.85
Epoch: 11 Train accuracy: 1.0 Test accuracy: 0.855
Epoch: 12 Train accuracy: 1.0 Test accuracy: 0.825
Epoch: 13 Train accuracy: 1.0 Test accuracy: 0.86
Epoch: 14 Train accuracy: 1.0 Test accuracy: 0.855


### Regulariazation for preventing overfit problem


In [ ]:
L1 = tf.contrib.layers.batch_norm(L1)
L2 = tf.contrib.layers.batch_norm(L2)

###  Tensorboard !

In [31]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "C:/Users/CSH/Desktop/tobigs" # 위치 바꿔라 위치링크
logdir = "{}/run-{}/".format(root_logdir, now)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
mse_summary = tf.summary.scalar('MSE', accuracy)

In [60]:
sess.close()